In [1]:
import os

In [2]:
%pwd

'd:\\recommendation-engine\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\recommendation-engine'

In [5]:
import pandas as pd

In [6]:
bookings = pd.read_csv("artifacts/data_ingestion/generated_data/bookings.csv")
comments = pd.read_csv("artifacts/data_ingestion/generated_data/comments.csv")
likes = pd.read_csv("artifacts/data_ingestion/generated_data/likes.csv")
bookings.head()
comments.head()
likes.head()

,user_id,liked_event_id,liked_event_organizer_id,like_id
0,dd273811-54c2-4c13-b0a5-fca4f3b621b8,66fa3a5d-fb3d-4faf-a295-2b7456f18620,1aafbaa0-9efd-4d51-adbb-310e8a3355d5,12f33ebf-2424-43b6-a72d-d8bcdc9ce806
1,09c23f75-ed99-49e7-8a34-2d9eb0c7f991,87ab34fa-e294-4f40-a51a-546079da2463,53da9a59-b5be-4ce2-bd73-e79826611b1c,e6e31bc3-219d-49e2-8344-3bb30b9e5fcc
2,f81fffe9-8813-48f9-87dc-01853f46e603,6b660a1b-07da-4c64-876e-03542029ed15,fcd1270d-0dd3-41ee-b999-7d39768c3369,cb4e26e2-76fa-4042-bb7f-2433dcf4ca2f
3,bd4d04b9-98cc-4937-bf84-f3e092a30758,39def82b-1f0a-4278-9d37-1ce25daf4cae,d86d4720-3bf8-49f9-85e3-a2d1c37d4cce,97e6060e-83ed-4510-9dcd-01b16614b867
4,cec3a311-1115-4f88-bf7b-8f2e67b59196,ba8c9541-9d61-4163-bc64-d613685ef041,85ddc80b-ecf1-4895-b690-d1d3f23c9576,7cea1a78-d43b-4120-bab5-159fc832001b


In [7]:
bookings.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   user_id                    150000 non-null  object
 1   booked_event_id            150000 non-null  object
 2   booked_event_organizer_id  150000 non-null  object
 3   booking_id                 150000 non-null  object
dtypes: object(4)
memory usage: 4.6+ MB


In [8]:
comments.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245397 entries, 0 to 245396
Data columns (total 4 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   user_id                       245397 non-null  object
 1   commented_event_id            245397 non-null  object
 2   commented_event_organizer_id  245397 non-null  object
 3   comment_id                    245397 non-null  object
dtypes: object(4)
memory usage: 7.5+ MB


In [9]:
likes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145397 entries, 0 to 145396
Data columns (total 4 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   user_id                   145397 non-null  object
 1   liked_event_id            145397 non-null  object
 2   liked_event_organizer_id  145397 non-null  object
 3   like_id                   145397 non-null  object
dtypes: object(4)
memory usage: 4.4+ MB


In [ ]:
bookings = bookings.rename(columns={
    'booked_event_id': 'event_id',
    'booked_event_organizer_id': 'organizer_id',
    'booking_id': 'id'
})

comments = comments.rename(columns={
    'commented_event_id': 'event_id',
    'commented_event_organizer_id': 'organizer_id',
    'comment_id': 'id'
})

likes = likes.rename(columns={
    'liked_event_id': 'event_id',
    'liked_event_organizer_id': 'organizer_id',
    'like_id': 'id'
})

bookings_df = bookings[['user_id', 'event_id', 'organizer_id']].copy()
bookings_df['is_booked'] = 1
bookings_df['num_bookings'] = 1

comments_df = comments[['user_id', 'event_id', 'organizer_id']].copy()
comments_df['is_commented'] = 1
comments_df['num_comments'] = 1

likes_df = likes[['user_id', 'event_id', 'organizer_id']].copy()
likes_df['is_liked'] = 1
likes_df['num_likes'] = 1

unique_pairs = pd.concat([
    bookings[['user_id', 'event_id']],
    comments[['user_id', 'event_id']],
    likes[['user_id', 'event_id']]
]).drop_duplicates()

if len(unique_pairs) > 200000:
    unique_pairs = unique_pairs.sample(n=200000, random_state=42)

organizer_mapping = pd.concat([
    bookings[['event_id', 'organizer_id']],
    comments[['event_id', 'organizer_id']],
    likes[['event_id', 'organizer_id']]
]).groupby('event_id')['organizer_id'].agg(lambda x: x.mode()[0]).to_dict()

result = unique_pairs.copy()

result = result.merge(
    bookings_df.groupby(['user_id', 'event_id']).agg({
        'is_booked': 'max',
        'num_bookings': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

result = result.merge(
    comments_df.groupby(['user_id', 'event_id']).agg({
        'is_commented': 'max',
        'num_comments': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

result = result.merge(
    likes_df.groupby(['user_id', 'event_id']).agg({
        'is_liked': 'max',
        'num_likes': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

for col in ['is_booked', 'num_bookings', 'is_commented', 'num_comments', 'is_liked', 'num_likes']:
    result[col] = result[col].fillna(0).astype(int)

result['organizer_id'] = (result['organizer_id_x']
                         .fillna(result['organizer_id_y'])
                         .fillna(result['organizer_id']))

result['organizer_id'] = result['organizer_id'].fillna(result['event_id'].map(organizer_mapping))

final_data = result[[
    'user_id', 
    'event_id', 
    'is_booked',
    'num_bookings',
    'is_liked',
    'num_likes',
    'is_commented', 
    'num_comments', 
    'organizer_id'
]]

final_data = final_data.drop_duplicates().sample(n=min(200000, len(final_data)), random_state=42)

final_data.to_csv("artifacts/data_ingestion/generated_data/merged_interactions.csv", index=False)

print("Merged data saved successfully!")
print(f"Final dataset shape: {final_data.shape}")
print(final_data.head())

Merged data saved successfully!
Final dataset shape: (200000, 9)
                                     user_id  \
119737  342a936b-14d4-4122-b40f-ee5e0968ab91   
72272   ca40fc77-06e3-4c04-b0d1-c8ed0a9cb4aa   
158154  41f36f68-7c52-43bc-8932-a97c97b039b3   
65426   29bcc80f-52e2-4d72-8eba-85fa652a3b0b   
30074   664c0cac-dba2-40f3-b5ac-25e203aa0b1f   

                                    event_id  is_booked  num_bookings  \
119737  eeab2cd6-bb34-4f89-a0f3-02488069b2b0          0             0   
72272   6e94de22-28ad-47a2-b035-96828df991e3          0             0   
158154  59a6b2ee-46cf-4c38-89be-3b0e7e2cbcb6          1             4   
65426   9b3a174a-4bc4-46f3-b3c7-37d3d663d9ad          0             0   
30074   324d7093-4571-40a9-9032-128ce3e35e4f          0             0   

        is_liked  num_likes  is_commented  num_comments  \
119737         0          0             1             1   
72272          0          0             1             3   
158154         0          0   